# Do Vector Search with DataStax on Cassandra

In the previous Chapter, we created a new database of movies with vectorized plots.

In this Chapter, we will search for movies which are close to the input query.

In [3]:
import cohere
from cassandra import ConsistencyLevel
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster

# get free Trial API Key at https://cohere.ai/
from cred import API_key

co = cohere.Client(API_key)

In [5]:
from cred import (ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET,
                  SECURE_CONNECT_BUNDLE_PATH)

KEYSPACE_NAME = "demo"

cloud_config = {"secure_connect_bundle": SECURE_CONNECT_BUNDLE_PATH}
auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.execute(f"USE {KEYSPACE_NAME};")

# 35K Movies DB

In [26]:
TABLE_NAME = "movies_35K_vectorized"

In [28]:
def findMovie(text, method="plot_vector_1024", limit=2):
    response = co.embed(model="embed-english-light-v2.0", texts=[text])
    vec = response.embeddings[0]
    for row in session.execute(
    f"SELECT title, wiki_link, plot FROM {KEYSPACE_NAME}.{TABLE_NAME} "
    f" ORDER BY {method} ANN OF %s LIMIT {limit}"
    , [vec]
):
        print(row.title, row.wiki_link)
        print(row.plot)
        print("--------------------------------------------------------")

In [29]:
findMovie("fictional beautiful animals")

Beauty and the Beast https://en.wikipedia.org/wiki/Beauty_and_the_Beast_(2017_film)
A beautiful enchantress disguised as an old beggar woman arrives at a castle during a ball and offers the host, a cruel and selfish prince, a rose in return for shelter from a storm. When he refuses, she reveals her identity. To punish the prince for his selfishness, the enchantress transforms him into a beast and his servants into household objects, then erases the castle, the prince and his servants from the memories of their loved ones. She casts a spell on the rose and warns the prince that the curse will only be broken if he learns to love another, and earn their love in return, before the last petal falls.
Some years later, in the small town of Villeneuve, Belle, the book-loving daughter of a music box maker and artist Maurice, dreams of adventure and brushes off advances from Gaston, an arrogant former soldier. On his way to a convention and lost in the forest, Maurice seeks refuge in the Beast's

In [30]:
findMovie("people live in dream")

Dream https://en.wikipedia.org/wiki/Dream_(2012_Telugu_film)
The story runs around a retired Army officer who did not get to fight the Kargil war. He now lives with his wife in an upscale, gated community spending most of his time playing war video games. The hero suffers from Hypnagogia, a rare medical disorder. He starts dreaming and believes that his dreams are real.
There is a story running in his dream and another in his real life. In a particular stage he fails to see the demarcation and these subconscious thoughts play havoc in his life and the people around him. Dream is this journey into the mind of the protagonist. This is a Medical thriller film.
--------------------------------------------------------
Dream Factory https://en.wikipedia.org/wiki/Dream_Factory_(film)
Four friends in Beijing—three men and a woman—start a company which specializes in fulfilling their clients' dreams. Enacting their wildest fantasies, they encounter many oddball clients along the way.
----------

# Top 250 IMDB

I also tried to convert plots of top 250 IMDB movies. However, the plots were only up to 2017.


In [33]:
TABLE_NAME = "movies_vectorized" # Top 250 IMDB

In [34]:
findMovie("people live in dream", "item_vector_1024")

Inception https://en.wikipedia.org/wiki/Inception
Dominick "Dom" Cobb and Arthur are "extractors", who perform corporate espionage using an experimental military technology to infiltrate the subconscious of their targets and extract valuable information through a shared dream world. Their latest target, Japanese businessman Saito, reveals that he arranged their mission himself to test Cobb for a seemingly impossible job: planting an idea in a person's subconscious, or "inception". To break up the energy conglomerate of ailing competitor Maurice Fischer, Saito wants Cobb to convince Fischer's son and heir, Robert, to dissolve his father's company. In return, Saito promises to use his influence to clear Cobb of a murder charge, allowing Cobb to return home to his children. Cobb accepts the offer and assembles his team: Eames, a conman and identity forger; Yusuf, a chemist who concocts a powerful sedative for a stable "dream within a dream" strategy; and Ariadne, an architecture student t

In [ ]:
# Close connection to Cassandra
cluster.shutdown()